*Unit 4, Sprint 1, Module 2*

---

# Vector Representations (Prepare)


As we learned yesterday, machines cannot interpret raw text. We need to transform that text into something we/machines can more readily analyze. Yesterday, we did simple counts of counts to summarize the content of Amazon reviews. Today, we'll extend those concepts to talk about vector representations such as Bag of Words (BoW) and word embedding models. We'll use those representations for search, visualization, and prepare for our classification day tomorrow.

Processing text data to prepare it for machine learning models often means translating the information from documents into a numerical format. Bag-of-Words approaches (sometimes referred to as Frequency-Based word embeddings) accomplish this by "vectorizing" tokenized documents. This is done by representing each document as a row in a DataFrame and creating a column for each unique word in the corpora (group of documents). The presence or lack of a given word in a document is then represented either as a raw count of how many times a given word appears in a document (CountVectorizer) or as that word's TF-IDF score (TfidfVectorizer).

On the python side, we will be focusing on `sklearn` and `spacy` today.  

## Case Study

We're going to pretend we're on the datascience team at the BBC. We want to recommend articles to visitors to on the BBC website based on the article they just read.

[BBC data set](http://mlg.ucd.ie/datasets/bbc.html)<br>
For today's exercise, we have downloaded the "raw text files" from the "BBC" data set and focused on the tech articles only

## Learning Objectives
* <a href="#p1">Part 1</a>: Represent a document as a vector
* <a href="#p2">Part 2</a>: Query Documents by Similarity
* <a href="#p3">Part 3</a>: Apply word embedding models to create document vectors

### Let's start with an analogy

![](https://peterbeshai.com/static/d9c3868cc1becd7648da453597a5d616/36dbb/DER_cube.jpg)

Pretend that the objecting floating in the room is our raw text dataset (i.e. a collection of documents).

A vectorizer is a mathematical transformation that takes our raw text data and transforms it into a numerical representation (i.e. numbers inside of vectors).

Depending on which vectorizer you use, you will be capturing some of the information encoded in the text but not other information.

So, as the analogy goes, depending which side of the floating object you stand and shine a light from, you will only see a portion of the information that exists in your text data set. Apply one vectorizer and you'll get word counts but not the contextual meaning; apply another vectorizer and you'll get the contextual meaning of the words but not the counts.

**Take Away:** Understand the benefits and limitations of using each of the vectorizers that we'll be learning today.

----

# 0. Colab notebook setup

##0.1 Get `spacy`

In [ ]:
%%time
# You'll use en_core_web_sm for the sprint challenge due memory constraints on Codegrader
#!python -m spacy download en_core_web_sm

# Locally (or on colab) let's use en_core_web_lg
!python -m spacy download en_core_web_md # Can do lg, takes awhile
# Also on Colab, need to restart runtime after this step!
#      or else Colab won't find spacy

##0.2 Restart the runtime!

## 0.3 Imports

In [ ]:
""" Import Statements """

# Classics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import spacy
nlp = spacy.load("en_core_web_md")

##0.4 Clone the git repo
so we can access the files in the `/data` folder

In [ ]:
!git clone https://github.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP.git

## 0.5 Get the BBC tech articles data

Helper function to read articles in `.txt` files in a specified directory and gather them into a big list

In [ ]:
import os

def gather_data(filefolder):
    """ Produces List of Documents from a Directory

    filefolder (str): a path of .txt files

    returns list of strings
    """

    data = []

    files = os.listdir(filefolder) # Causes variation across machines

    for filename in files:

        path = os.path.join(filefolder, filename)

        if  path[-3:] == 'txt': # os ~endswith('txt')
            with open(path, 'rb') as f:
                data.append(f.read())

    return data

Read the articles into a list

In [ ]:
# locate and get path to the /data folder, using colab's file browser
data_path = '/content/DS-Unit-4-Sprint-1-NLP/module2-vector-representations/data'
data = gather_data(data_path)
print(f'Number of Articles: {len(data)}')

Articles turn out to be "byte strings" so we need to decode them to strings

In [ ]:
print(data[20])
print(type(data[20]))

In [ ]:
# decode bytestrings in a corpus to strings
# takes a list of documents, i.e a list of strings as input
articles = []
for article in data:
  article = article.decode("utf-8")
  articles.append(article)
print(len(articles))

# Preview first article and check data type again
data = articles
print(data[0])
print(type(data[0]))

Make a `spacy` tokenizer, as in Module 1<br>
Question: why did we use [`.strip()`](https://python-reference.readthedocs.io/en/latest/docs/str/strip.html)?

In [ ]:
def tokenize(document):
    """
    Takes a doc (text string) and returns a list of tokens in the form of lemmas.
    Filters out Stop words, punctuation, and leading/trailing spaces.
    """

    doc = nlp(document)
    # last "and" statement is to remove token lemmas that are empty strings
    lemma_list = [token.lemma_.lower().strip() for token in doc if (not token.is_stop)
                                                                and (not token.is_punct )
                                                                and (token.lemma_.strip()!="")
                                                                and (len(token.lemma_.strip())>1)]
    return lemma_list

In [ ]:
# Test our tokenizer
tokenize('<Insert sentence here>')

In [ ]:
data[20]

In [ ]:
tokenize(data[20])

# 1. Represent a document as a vector (Learn)
<a id="p1"></a>

## Overview

In this section, we are going to create Document Term Matrices (DTM). <br>
In the DTM, each row represents a document. The columns correspond to the vocabulary words of the entire corpus of documents. <br>
Today, we'll learn three methods of computing the value in each cell. <br>
* `CountVectorizer` from `sklearn` provides a basic implementation: counts of appearances of words. You could also ignore multiple occurrences of words in a document, and just keep track of whether or not a word occurs in the document via a boolean value.<br>
* `TfidfVectorizer` is a more advanced implementation that keeps track of *term-frequency inverse-document frequency* (TF-IDF) instead of integer counts.

* Word embeddings, an even more sophisticated method that helps solve the problem of accounting for the context of a word.

**Discussion:** Don't we lose all the context and grammar if we do this? So Why does it work?

## Follow Along

**Warm Up (_3 Minutes_)**

Extract the tokens from this sentence using Spacy. Text is from [OpenAI](https://openai.com/blog/better-language-models/)

In [ ]:
text = """
GPT-2 displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality,
where we prime the model with an input and have it generate a lengthy continuation. In addition, GPT-2 outperforms other language models
trained on specific domains (like Wikipedia, news, or books) without needing to use these domain-specific training datasets. On language
tasks like question answering, reading comprehension, summarization, and translation, GPT-2 begins to learn these tasks from the raw text,
using no task-specific training data. While scores on these downstream tasks are far from state-of-the-art, they suggest that the tasks
can benefit from unsupervised techniques, given sufficient (unlabeled) data and compute."""

print(f'Number of Words: {len(text.split())}')

In [ ]:
tokens = tokenize(text)
print(f'Number of Words: {len(tokens)}')
tokens

-----
# 1.1 `CountVectorizer`

![](https://images4.programmersought.com/947/0a/0acb9279d17a1631bcfb154583cca443.JPEG)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# list of text documents

# create the transformer

# build vocab

# transform text

### BEGIN SOLUTION
# a corpus is a collection of text documents
corpus = ["We created a new dataset which emphasizes diversity of content, by scraping content from the Internet.",
          " In order to preserve document quality, we used only pages which have been curated/filtered by humans—specifically, we used outbound links from Reddit which received at least 3 karma.",
          " This can be thought of as a heuristic indicator for whether other users found the link interesting (whether educational or funny), leading to higher data quality than other similar datasets, such as CommonCrawl."]

# create the transformer


# build vocabulary


# transform text to create the document-term matrix


### END SOLUTION

In [ ]:
# explore the document-term matrix

# Get feature names

# print out raw text for comparison with  indexing

# show indexing of token count

# Dealing with Sparse Matrix

# Get Word Counts for each document

### BEGIN SOLUTION

# Get feature names -- this is the "vocablulary" for the corpus


In [ ]:
# explore the document-term matrix and understand it's structure


In [ ]:
# Get Word Counts for each document and format as a dataframe

### END SOLUTION

In [ ]:
dtm_new

**Three Minute Challenge:**
* Apply CountVectorizer to our BBC Data
* Store results in a dataframe called `dtm`
* Extra Challenge - Try to Customize CountVectorizer with Spacy Processing

In [ ]:
# Apply CountVectorizer to our Data
# Use custom Spacy Vectorizer
# BBC articles in `data` variable

from sklearn.feature_extraction.text import CountVectorizer

# Learn our Vocab
# Get sparse dtm (i.e. transform the data)

### BEGIN SOLUTION


# Learn our Vocab


# Transform the BBC news articles


In [ ]:
import seaborn as sns
sns.set()

doc_len = [len(doc.split()) for doc in data]
sns.distplot(doc_len);

#1.2  `TfidfVectorizer`

## Term Frequency - Inverse Document Frequency (TF-IDF)

![](https://miro.medium.com/max/1404/1*mu6G-cBmWlENS4pWHEnGcg@2x.jpeg)


**Term Frequency**: $\text{tf}_{i,j}$  is the number of times term $i$ appeared in document $j$

**Inverse Document Frequency:** A weight penalty for terms that exist in a high fraction of documents.

The purpose of TF-IDF is to find what is **unique** to each document. Because of this we will penalize the term frequencies of words that are common across all documents. This effectively upweights terms that are rarer across documents.


#1.2  `TfidfVectorizer`

## Term Frequency - Inverse Document Frequency (TF-IDF)

![](https://miro.medium.com/max/1404/1*mu6G-cBmWlENS4pWHEnGcg@2x.jpeg)


**Term Frequency**: $\text{tf}_{i,j}$  is the number of times term $i$ appeared in document $j$

**Inverse Document Frequency:** A weight penalty for terms that exist in a high fraction of documents.

The purpose of TF-IDF is to find what is **unique** to each document. Because of this we will penalize the term frequencies of words that are common across all documents. This effectively upweights terms that are rarer across documents.

<img src="https://upload.wikimedia.org/wikipedia/commons/8/81/Logarithm_plots.png" width="700" height="400" />

It's useful to reference both the algebraic and geometric representations of a single mathematical ideal whenever possible in order to build the fullest understanding possible.

The IDF portion of the TF-IDF equation has been coded up below so that we can play around with the values and get a better understanding of how this portion of the equation works.

NOTE: There are other ways to construct the equation for the IDF term; different constructions of the equation serve different purposes and which you ultimately use simply depends on your problem/task. You can check out the [wikipedia article on tfidf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

In practice, you usually don't even have to think about the mathematical formulation of the IDF term. You simply import the tfidf vectorizer and use its api. The rare exception to this would if you are working a very particular kind of problem where the open source implementation of tfidf doesn't suit your needs and so then you might consider creating your own equation or using a different one that you read about.

In [ ]:
# inverse document frequency score
# the plus ones are constants that shift the around the baseline value
def idf(n, df):
    """
    Parameters
    ----------
    n: int
        num of docs in corpus

    df: int
        num of docs that term t (i.e. a token) appears in

    Returns
    -------
    inverse docuemnt frequency: float
    """
    return np.log( n/ df)

In [ ]:
n = 100                 # num of docs in corpus
df_range = 100
IDF = []
DF = []
for df in range(1, df_range + 1):
    idf_score = idf(n, df)
    IDF.append(idf_score)
    DF.append(df)

In [ ]:
plt.figure()
plt.title("Inverse Document Frequency Score vs. Document Frequency of Token")
plt.ylabel("Inverse Document Frequency Score")
plt.xlabel("Document Frequency of Token")
plt.grid()
plt.plot(DF, IDF);
plt.show()

plt.figure()
plt.title("Inverse Document Frequency Score vs. Document Frequency of Token")
plt.ylabel("Inverse Document Frequency Score")
plt.xlabel("Document Frequency of Token")
plt.xlim(95,100)
plt.ylim(0,0.10)
plt.grid()
plt.plot(DF, IDF);
plt.show()

In [ ]:
# let's work through a couple of examples to help build intuition
# imagine the token we are considering is "purchase", which rarely appears
# imagine the token that we are considering is "the", which appears often

n = 100

# imagine the token we are considering is "purchase"
tf = 1
df = 20

tfidf_score = tf * idf(n, df)
print(f'purchase: {tfidf_score:.2f}')

# imagine the token that we are considering is "the"
tf = 50
df = 95

tfidf_score = tf * idf(n, df)
print(f'the: {tfidf_score:.2f}')

Now that we have built some intuition on how TFIDF works, let's use `sklearn`'s `TfidfVectorizer` to vectorize our dataset. <br>
Note that `TfidfVectorizer` is used with the same syntax as `CountVectorizer`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)

# 2. Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(data)

# 3. View term-document matrix as DataFrame
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())
print(dtm.shape)
dtm.head()

Now let's get a little fancier with the `TfidfVEctorizer`: we'll use the keywords `ngram_range`, `max_df`, `min_df` and `tokenizer` to accomplish the following:
- Build a vocabulary from unigrams and bigrams
- Do "statistical trimming" by ignoring all terms with document frequency higher than `max_df` and all terms with document frequency lower than `min_df`
- Use the `spaCy` tokenizer that we built

In [ ]:
%%time
# Tuning the Parameters of our Vectorizer

### BEGIN SOLUTION

# Instantiate vectorizer object (ngram_range includes unigrams and bigrams)
tfidf = TfidfVectorizer(...)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(data) # Similiar to fit_predict -- produces a sparse matrix

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())
print(dtm.shape)

### END SOLUTION

In [ ]:
dtm.head()

## Challenge

During this module's project assignment, you will transform data science job listings to vector representations for analysis downstream.

# 2. Query Documents by Similarity (Learn)
<a id="p2"></a>

## Overview

Have you ever considered how a search bar works in Google? You may just think that search bars simply match your input text against the documents. While there are many different mechanisms for the 'match', one of the most classic is to search by similarity. We will apply two measures of similarity between a document and the input query document: the **Cosine Similarity** between the documents and the **Euclidean distance** between the documents. <br>

Cosine Similarity is [more compute intensive](https://stackoverflow.com/questions/28917985/why-cosine-distance-is-much-slower-than-using-euclidean-distance-with-dbscan-alg) than Euclidean distance.

## Follow Along
*Cosine similarity* and *nearest-neighbor distance* provide two measures that can be used to gauge the similarity of one word to another. <br>
We develop these concepts in this section.

### 2.1 Cosine Similarity
The [`cosine_simlarity`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) between two feature vectors is the dot product of the *normalized* feature vectors. <br>To *normalize* a vector, you simply divide it by its length: thus<br><br>
$\text{cosine_similarity}(\textbf{u},\textbf{v})= \frac{\textbf{u}}{\left\|\textbf{u}\right\|} \cdot \frac{\textbf{v}}{\left\|\textbf{v}\right\|}$, <br><br>
where $\left\|\textbf{u}\right\|$ is the length of the vector $\textbf{u}$<br><br>

$\textbf{cosine_similarity}$ is in the interval $[0, 1]$, <br><br>
The larger the $\textbf{cosine_similarity}$, the more similar the words are, and vice-versa.<br><br>

`cosine_similarity` takes as input the document-term matrix <br>
and creates a **similarity matrix** in which the rows and columns represent documents <br>
and entry ($i, j$) is the cosine similarity between documents $i$ and $j$.<br><br>

Let's compute the **similarity matrix** for the BBC tech articles corpus<br>



In [ ]:
%%time
from sklearn.metrics.pairwise import cosine_similarity

# Calculate Distance of TF-IDF Vectors

# Turn it into a DataFrame

# Our Similarity Matrix is ? size

# Each row is the similarity of one document to all other documents (including itself)

# Grab the row and pick off the indicies of the most/least similar docs

# Calculate Distance of TF-IDF Vectors
from sklearn.metrics.pairwise import cosine_similarity

# similarity matrix
dist_matrix = cosine_similarity(dtm)

# Turn it into a DataFrame
df = pd.DataFrame(dist_matrix)

# Size of the Similarity Matrix
print(df.shape)

In [ ]:
df.head()

In [ ]:
# Each row is the similarity of one document to all other documents (including itself)
df.iloc[0]

### Find the most similar documents to the first document

In [ ]:
### BEGIN SOLUTION
# Grab the first row and return the indices of the 5 most similar docs

### END SOLUTION

### 2.2 K Nearest Neighbors Search (kNN)

<img src="https://miro.medium.com/max/753/0*jqxx3-dJqFjXD6FA" width="600" height="400" />


Nearest Neighbor models are distance based algorithms. They store your training set in memory.

### `scikit-learn`'s `NearestNeighbors` class
computes distances between all pairs of data points, <br>
and returns the result in the form of a similarity matrix. <br>
Entry $(i,j)$ in the similarity matrix records the similarity of data points $i$ and $j$.<br>

Below is an example using `NearestNeighbors` to find the most similar documents to a given documentL


In [ ]:
%%time
from sklearn.neighbors import NearestNeighbors

# Instantiate the nearest neighbors model
nn = NearestNeighbors()

# fit to our document term matrix
nn.fit(dtm.values)

Find the 5 most similar documents to a query document; here we will choose the first document as our query document

In [ ]:
# sample a doc from a dtm to use as our query point
doc_index = 0
query_doc = dtm.values[0][None,:] # re-cast as a row vector

# Query Using the kneighbors method
# NOTE: nn counts the original document as one of the neighbors, so if we want 5 nearest neighbors, we should set n_neightbors=6
neigh_dist, neigh_index = nn.kneighbors(query_doc,n_neighbors=6)

In [ ]:
# nearest neighbor search returns the same documents as cosine similarity!
print(neigh_index)
print(neigh_dist)

Which words do these two documents have in common?

In [ ]:
def shared_words(s1, s2):
  res = []
  l_s1, l_s2 = set(s1.split()), set(s2.split())
  for ss1 in l_s1:
    if ss1 in l_s2: res.append(ss1)
  return res

doc1_tokens = tokenize(data[0])
doc2_tokens = tokenize(data[...])
shared_words(' '.join(doc1_tokens), ' '.join(doc2_tokens))

### Exercise:
Search the BBC dataset for documents most similar to a random query doc

In [ ]:
rndm_tech_article = [ """
Blockchain technology encompasses so much more than just cryptocurrencies these days. And while these currencies are by far blockchain’s most famous use case, the potential for blockchain far, far exceeds a straightforward transfer of value.
The gaming industry, for example, has always been fairly controversial due to a range of issues. Many players feel particularly worried about the protection of their data, fraudulent activities, high fees, and most of all, unfair and hidden odds.
In these cases, P2P doesn’t just mean peer-to-peer; it means player-to-player. As with the other P2P industries that blockchain is disrupting, such as FinTech, there are already plenty of ways in which blockchain technology is being applied to resolve these issues and many more of the challenges faced by the gaming industry. Here are just a few.
Reducing fraud
The gaming industry suffers a lot from online fraud. One of the main advantages of blockchain technology is that it enables highly secure data encryption. What’s more, all of it is entirely accessible and transparent to the player. In other words, blockchain is a for game changer for venues such as online casinos, lotteries, and virtually anything which relies on random number generation.
On top of this, hackers will have particularly hard time, if it is even possible, to destroy a decentralized blockchain network, making sure gamer data stays safe. This is an inherent feature of distributed ledger technology whereby nodes in the ledger maintain the distributed databases in a shared manner, and each node has the complete information in the entire database.
"""]


# transform the rndm_tech_article to get its document vector


In [ ]:
# use k Nearest Neighbors to find the most similar documents in our document term matrix to the query


In [ ]:
# Inspect the most similar documents


## Challenge

In the module project assignment, you will apply one of these search techniques to retrieve documents related to a query document.

# 3. Apply word embedding models to create document vectors (Learn)
<a id="p3"></a>


## Overview
### Bag of Words (BoW) models lose information about context

One of the limitations of Bag-of-Words approaches is that any information about the context surrounding that word is lost. This also means that with bag-of-words approaches often the only tools that we have for identifying words with similar usage or meaning and subsequently consolidating them into a single vector is through the processes of stemming and lemmatization which tend to be quite limited at consolidating words unless the two words are very close in their spelling or in their root parts-of-speech.

### Embedding approaches preserve more textual context
Word2Vec is an increasingly popular word embedding technique. Like Bag-of-words it learns a real-value vector representation for a predefined fixed-size vocabulary that is generated from a corpus of text. However, in contrast to BoW, Word2Vec approaches are much more capable of accounting for textual context, and are better at discovering words with similar meanings or usages (semantic or syntactic similarity).

##3.1 Word2Vec Intuition
Reference: [Deep NLP: Word Vectors with Word2Vec](https://medium.com/deep-learning-demystified/deep-nlp-word-vectors-with-word2vec-d62cb29b40b3)
### The Distribution Hypothesis

In order to understand how Word2Vec preserves textual context we have to understand what's called the [Distribution Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics). The Distribution Hypothesis operates under the assumption that words that have similar contexts will have similar meanings. Practically speaking, this means that if two words are found to have similar words both to the right and to the left of them throughout the corpora then those words have the same context and are assumed to have the same meaning.

> "You shall know a word by the company it keeps" - John Firth

This means that we let the usage of a word define its meaning and its "similarity" to other words. In the following example, which words would you say have a similar meaning?

**Sentence 1**: Traffic was light today

**Sentence 2**: Traffic was heavy yesterday

**Sentence 3**: Prediction is that traffic will be smooth-flowing tomorrow since it is a national holiday

What words in the above sentences seem to have a similar meaning if all you knew about them was the context in which they appeared above?

Lets take a look at how this might work in action, the following example is simplified, but will give you an idea of the intuition for how this works.

#### Corpora:

1) "It was the sunniest of days."

2) "It was the rainiest of days."

#### Vocabulary:

{"it": 1, "was": 2, "the": 3, "of": 4, "days": 5, "sunniest": 6, "rainiest": 7}

### Vectorization

|       doc   | START_was | it_the | was_sunniest | the_of | sunniest_days | of_it | days_was | it_the | was_rainiest | rainiest_days | of_END |
|----------|-----------|--------|--------------|--------|---------------|-------|----------|--------|-------------|--------------|--------|
| it       | 1         | 0      | 0            | 0      | 0             | 0     | 1        | 0      | 0           | 0            | 0      |
| was      | 0         | 1      | 0            | 0      | 0             | 0     | 0        | 1      | 0           | 0            | 0      |
| the      | 0         | 0      | 1            | 0      | 0             | 0     | 0        | 0      | 1           | 0            | 0      |
| sunniest | 0         | 0      | 0            | 1      | 0             | 0     | 0        | 0      | 0           | 0            | 0      |
| of       | 0         | 0      | 0            | 0      | 1             | 0     | 0        | 0      | 0           | 1            | 0      |
| days     | 0         | 0      | 0            | 0      | 0             | 0     | 0        | 0      | 0           | 0            | 1      |
| rainiest  | 0         | 0      | 0            | 1      | 0             | 0     | 0        | 0      | 0           | 0            | 0      |

Each column vector represents the word's context -in this case defined by the words to the left and right of the center word. How far we look to the left and right of a given word is referred to as our "window of context." Each row vector represents the the different usages of a given word. Word2Vec can consider a larger context than only words that are immediately to the left and right of a given word, but we're going to keep our window of context small for this example. What's most important is that this vectorization has translated our documents from a text representation to a numeric one in a way that preserves information about the underlying context.

We can see that words that have a similar context will have similar row-vector representations, but before looking that more in-depth, lets simplify our vectorization slightly. You'll notice that we're repeating the column-vector "it_the" twice. Lets combine those into a single vector by adding them element-wise.

|       *   | START_was | it_the | was_sunniest | the_of | sunniest_days | of_it | days_was | was_rainiest | rainiest_days | of_END |
|----------|-----------|--------|--------------|--------|---------------|-------|----------|-------------|--------------|--------|
| it       | 1         | 0      | 0            | 0      | 0             | 0     | 1        | 0           | 0            | 0      |
| was      | 0         | 2      | 0            | 0      | 0             | 0     | 0        | 0           | 0            | 0      |
| the      | 0         | 0      | 1            | 0      | 0             | 0     | 0        | 1           | 0            | 0      |
| sunniest | 0         | 0      | 0            | 1      | 0             | 0     | 0        | 0           | 0            | 0      |
| of       | 0         | 0      | 0            | 0      | 1             | 0     | 0        | 0           | 1            | 0      |
| days     | 0         | 0      | 0            | 0      | 0             | 0     | 0        | 0           | 0            | 1      |
| rainiest  | 0         | 0      | 0            | 1      | 0             | 0     | 0        | 0           | 0            | 0      |

Now, can you spot which words have a similar row-vector representation? Hint: Look for values that are repeated in a given column. Each column represents the context that word was found in. If there are multiple words that share a context then those words are understood to have a closer meaning with each other than with other words in the text.

Lets look specifically at the words sunniest and rainiest. You'll notice that these two words have exactly the same 10-dimensional vector representation. Based on this very small corpora of text we would conclude that these two words have relatd meanings because they share the same usage. Is this a good assumption? Well, they are both referring to the weather outside so that's better than nothing. You could imagine that as our corpora grows larger we will be exposed a greater number of contexts and the Distribution Hypothesis assumption will improve.

##3.2 Word2Vec Variants

###3.2.1 Skip-Gram

The Skip-Gram method predicts the neighbors’ of a word given a center word. In the skip-gram model, we take a center word and a window of context (neighbors) words to train the model and then predict context words out to some window size for each center word.

This notion of “context” or “neighboring” words is best described by considering a center word and a window of words around it.

For example, if we consider the sentence **“The speedy Porsche drove past the elegant Rolls-Royce”** and a window size of 2, we’d have the following pairs for the skip-gram model:

**Text:**
**The**	speedy	Porsche	drove	past	the	elegant	Rolls-Royce

*Training Sample with window of 2*: (the, speedy), (the, Porsche)

**Text:**
The	**speedy**	Porsche	drove	past	the	elegant	Rolls-Royce

*Training Sample with window of 2*: (speedy, the), (speedy, Porsche), (speedy, drove)

**Text:**
The	speedy	**Porsche**	drove	past	the	elegant	Rolls-Royce

*Training Sample with window of 2*: (Porsche, the), (Porsche, speedy), (Porsche, drove), (Porsche, past)

**Text:**
The	speedy	Porsche	**drove**	past	the	elegant	Rolls-Royce

*Training Sample with window of 2*: (drove, speedy), (drove, Porsche), (drove, past), (drove, the)

The **Skip-gram model** is going to output a probability distribution i.e. the probability of a word appearing in context given a center word and we are going to select the vector representation that maximizes the probability.

With CountVectorizer and TF-IDF the best we could do for context was to look at common bi-grams and tri-grams (n-grams). Well, skip-grams go far beyond that and give our model much stronger contextual information.

![alt text](https://www.dropbox.com/s/c7mwy6dk9k99bgh/Image%202%20-%20SkipGrams.jpg?raw=1)

###3.2.2 Continuous Bag of Words

This model takes thes opposite approach from the skip-gram model in that it tries to predict a center word based on the neighboring words. In the case of the CBOW model, we input the context words within the window (such as “the”, “Porsche”, “drove”) and aim to predict the target or center word “speedy” (the input to the prediction pipeline is reversed as compared to the SkipGram model).

A graphical depiction of the input to output prediction pipeline for both variants of the Word2vec model is attached. The graphical depiction will help crystallize the difference between SkipGrams and Continuous Bag of Words.

![alt text](https://www.dropbox.com/s/k3ddmbtd52wq2li/Image%203%20-%20CBOW%20Model.jpg?raw=1)

###3.2.3 Notes on Word Embeddings:

1) Word2Vec is useful for topic-modeling, because each word vector contains information about related context words.

2) Word2Vec can result in really large and complex vectorizations. In fact, you need Deep Neural Networks to train your Word2Vec models from scratch, but we can use helpful pretrained embeddings to do really cool things!
Let's take a look at how to work with these word vectors.

## 3.3 Spacy word embeddings
#### `spacy` provides pretrained Word2Vec models.

We will use their `en_core_web_md` model, which has 300-dimensional word embeddings for 20,000 words.

In [ ]:
# Process a text
doc = nlp("bananas")

# Get the vector for the token "bananas"
bananas_vector = doc.vector

# print the dimensionality of the vector embedding
print(len(bananas_vector))
print(bananas_vector)

In [ ]:
apple = nlp('apple')
banana = nlp('banana')
gravity = nlp('gravity')

print(f'Apple -> Banana similarity: {apple.similarity(banana)}')
print(f'Apple -> Gravity similarity: {apple.similarity(gravity)}')
print(f'Banana -> Gravity similarity: {banana.similarity(gravity)}')

### Spacy Document vectors

From spacy we can get word2vec embeddings for each word in a document<br>
We can create a document vector by averaging all the word vectors in that doc<br>
We can calculate the similarity between documents now!

In [ ]:
doc1 = nlp("On a hot summer day, it's good to drink water.")
doc2 = nlp("There are lots of cold drinks in the refrigerator.")
doc3 = nlp("cosine distance measures similarity")

In [ ]:
len(doc1.vector)

In [ ]:
type(doc)

In [ ]:
# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

In [ ]:
doc3.similarity(doc2)

In [ ]:
doc3.similarity(doc1)

## Challenge: can we make a "thesaurus"?<br>
Given a query word, do you think we could find good synonyms  <br>
by doing a nearest neighbor search on the `spacy` word embeddings?<br>
Let's give it a try!

First, get the embeddings for the vocabulary words and make a document-term matrix

In [ ]:
%%time
vocab = list(nlp.vocab.strings)
print(len(vocab))

In [ ]:
# eliminate duplicates due to mixing upper and lower case
vocab =[word.lower() for word in vocab]
vocab = list(set(lower_vocab))
print(len(vocab))

In [ ]:
%%time
word_vectors = [nlp.vocab.get_vector(word) for word in vocab]
word_vectors = np.asarray(word_vectors)
print(word_vectors.shape)

Fit a nearest-neighbors model to the document-term matrix

In [ ]:
%%time
nn = NearestNeighbors(n_neighbors=10, algorithm='auto')

# fit to the document-term matrix of all the vocabulary words
nn.fit(word_vectors)

Choose a query word, for which you want to find synonyms

In [ ]:
query_word = 'marvelous'
query_vector = nlp.vocab.get_vector(query_word)[None,:]
query_vector.shape

Find the nearest neighbors to the query, from spacy's vocabulary

In [ ]:
neigh_dist, neigh_index = nn.kneighbors(query_vector, n_neighbors=20)
[vocab[int(index)] for index in neigh_index[0]]

## Can we visualize word vectors?

300 dimensional word vectors are difficult (impossible) to visualize! <br>
However, we can use Principal Components Analysis (PCA) to reduce their dimensionality. <br>

PCA is one of the most important techniques in machine learning, and indeed in all of mathematics! PCA transforms the feature vectors of a data set to their "Principal Components", which are ranked in order of their importance in describing the data set. By eliminating the less important Principal Components, we can reduce the dimensionality of the data set, at the cost of losing some information.

If you're interested to explore this topic in greater depth, here are a couple of references:
- One of the best brief introductions to the mathematics of PCA is [Daniela Witten's tweetstorm](https://twitter.com/womeninstat/status/1285610321747611653?lang=en)
- For a more in-depth, but still understandable exposition of PCA, watch the Statquest video [Principal Component Analysis (PCA), Step-by-Step](https://youtu.be/FgakZw6K1QQ) from Josh Starmer.

Now let's use PCA to reduce our 300-dimensional word vectors down to two dimensions, so that we can visualize them in a plane. Of course, in the process we lose a lot of information, but these 2D word vectors still pack a surprising amount of information!

In [ ]:
# import the PCA module from sklearn
from sklearn.decomposition import PCA

def get_word_vectors(words):
    # converts a list of words into their word vectors
    return [nlp(word).vector for word in words]

words = ['machine learning', 'man', 'woman',  'king', 'queen',
         'artificial intelligence', 'nurse', 'doctor',
         'data', 'science', 'concrete', 'wood',  'marble',
         'design', 'color', 'font']


word_vectors = get_word_vectors(words)


# intialise pca model and tell it to project data down onto 2 dimensions

# fit the pca model to our 300D data, this will work out which is the best
# way to project the data down that will best maintain the relative distances
# between data points. It will store these intructioons on how to transform the data.

# Tell our (fitted) pca model to transform our 300D data down onto 2D using the
# instructions it learnt during the fit phase.

# let's look at our new 2D word vectors

# intialise pca model and tell it to project data down onto 2 dimensions
pca = PCA(n_components=2)

# fit the pca model to our 300D data, this will work out which is the best
# way to project the data down that will best maintain the relative distances
# between data points. It will store these intructioons on how to transform the data.
pca.fit(word_vectors)

# Tell our (fitted) pca model to transform our 300D data down onto 2D using the
# instructions it learnt during the fit phase.
word_vecs_2d = pca.transform(word_vectors)

# let's look at our new 2D word vectors
word_vecs_2d

In [ ]:
# create a nice big plot
plt.figure(figsize=(12,8))

# plot the scatter plot of where the words will be
plt.scatter(word_vecs_2d[:,0], word_vecs_2d[:,1])

# for each word and coordinate pair: draw the text on the plot
for word, coord in zip(words, word_vecs_2d):
    x, y = coord
    plt.text(x, y, word, size= 15)

# show the plot
plt.grid();
plt.show()

### Spend a few minutes exploring the magic of word vectors!
Create your own `words` list and run it through the code in the above two cells. <br>
Perhaps include clusters of words that have similar meanings or that might <br>reveal implicit gender bias or some other kind of bias. What did you find out?

## Challenge

You will extract word embeddings from documents using Spacy's pre-trained model in the upcoming module project.

## King - Man + Woman = Queen

Check out [**The amazing power of word vectors**](https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/), which explores the above equation

# Sources

* Spacy 101 - https://course.spacy.io
* NLTK Book - https://www.nltk.org/book/
* An Introduction to Information Retrieval - https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf